In [3]:
import polars as pl
import numpy as np
import os
import pandas as pd

from torch.utils.data import DataLoader
from transformers import AdamW
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split


In [4]:
# !pip install polars

In [5]:
dial_train_path = 'dial_train.parquet'
target_train_path = 'train_target.parquet'

In [6]:
target_train = pd.read_parquet(target_train_path)
target_train.mon = pd.to_datetime(target_train.mon)

dial_train = pd.read_parquet(dial_train_path)

In [7]:
dial_train

,client_id,event_time,embedding,mon
0,b27b9c54e72728e7bbfbe96ef2f3d49c14c9c5f0900033...,2022-01-12 12:22:28.151649,"[0.34171295, -0.052265663, 0.5509638, -0.25280...",1
1,bff7260208097c052cea083ddc9e961a8d23c1b1c2b268...,2022-01-15 10:19:36.296643,"[0.25192896, -0.057982136, 0.56129426, -0.2468...",1
2,c977ed2889aacd9aa35420cce5652274b1a1d347648d80...,2022-01-20 11:56:28.229993,"[0.10494808, 0.12875096, 0.34333166, 0.1237644...",1
3,d2e003fda662d4362aed928dea8bdaffaaac002e6cc435...,2022-01-08 09:54:14.315763,"[0.34184495, -0.0066548246, 0.41655463, -0.361...",1
4,d887ecc28f596b1ccf4d9758c1974d2c3058041082ed6a...,2022-01-11 06:19:57.134893,"[0.20899382, -0.20362832, 0.50806755, -0.37998...",1
...,...,...,...,...
206238,3b28346c9687dc7b7f293f9a232b3372fa0564cacf520b...,2022-09-23 18:31:02.682112,"[0.3919262, -0.042448077, 0.5407961, -0.203592...",9
206239,3b28346c9687dc7b7f293f9a232b3372fa0564cacf520b...,2022-09-18 11:01:40.032310,"[0.49427247, -0.37785238, 0.48056543, -0.49872...",9
206240,3b28346c9687dc7b7f293f9a232b3372fa0564cacf520b...,2022-09-12 14:08:59.343379,"[0.0058998824, -0.16908917, 0.3124224, -0.1694...",9
206241,3b28346c9687dc7b7f293f9a232b3372fa0564cacf520b...,2022-09-15 08:29:21.048320,"[0.28633767, 0.039378952, 0.4970856, -0.255222...",9


In [ ]:
def amount_agregate(tg, trx_timeseries, geo_timeseries):

    feats = []
    for mon in tg.mon.unique():
        trx = trx_timeseries[trx_timeseries.event_time < mon]
        geo = geo_timeseries[geo_timeseries.event_time < mon]
        feats_mon = trx.groupby('client_id')[trxs].mean().reset_index()
        feats_mon2 = geo.groupby('client_id')[hashes].mean().reset_index()
        feats_mon['mon'] = mon
        for hash in hashes:
            feats_mon[hash] = feats_mon2[hash]
        
        feats.append(feats_mon)
        print(mon)

    feats = pd.concat(feats, axis=0)
    return feats

feats = amount_agregate(target_train, trx_train, geo_train)

In [18]:
len(target_train)

2049336

In [20]:
# embeddings = []
# # labels = []

# for i in range(len(target_train)):
#     embeds = dial_train[(dial_train['client_id'] == target_train.iloc[i, 5]) \
#     & (dial_train['event_time'] < target_train.iloc[i, 0])]
#     if len(embeds) > 0:
#         sum_emb = embeds.groupby('client_id')['embedding'].sum()[0]
#     else:
#         sum_emb = np.zeros(len_emb)
        
#     embeddings.append(sum_emb)
#     if i % 10000 == 0:
#         print(i)
#     # labels.append(target_train.iloc[i, 1:5])

In [19]:
ids = target_train.client_id.values

In [22]:
train_ids, val_ids = train_test_split(ids, test_size=.2)

In [23]:
class dataset(Dataset):
    def __init__(self, ids, dial_train, target_train):
        self.df = target_train[target_train.client_id.isin(ids)]
        self.dial_train = dial_train
        self.target_train = target_train
    
    def get_emb(self, row):
        embeddings = self.dial_train[(self.dial_train['client_id'] == row.client_id) \
        & (self.dial_train['event_time'] < row.mon)]
        if len(embeddings) > 0:
            sum_embedding = embeddings.groupby('client_id')['embedding'].sum()[0]
        else:
            sum_embedding = np.zeros(len_emb)
                
        labels = row[1:5]
            
        return labels, sum_embedding

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        labels, embeds = self.get_emb(row)
        item = {}
        item['summary_embeddings'] = torch.tensor(embeds)        
        item['labels'] = torch.tensor(labels)
        
        return item

    def __len__(self):
        return len(self.df)

train_dataset = dataset(train_ids, dial_train, target_train)
val_dataset = dataset(val_ids, dial_train, target_train)

In [24]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

### Model FC

In [27]:
## training parameters to be used for all models ##
num_train_epochs = 5
batch_size = 16
learning_rate = 1.0e-5
weight_decay = 0.01
warmup_steps = 0
max_seq_length = 64

In [28]:
import json
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoModel, AutoTokenizer, get_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from tqdm.notebook import tqdm, trange
from time import perf_counter
from PIL import Image
import pandas as pd

In [29]:
# set random seeds for repeatability
import numpy as np
import random

def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

In [83]:
class VLBertModel(torch.nn.Module):

    def __init__(self, num_labels, hidden_states):
        super().__init__()

        self.num_labels = num_labels
        self.hidden_states = hidden_states
        self.classifier = nn.Linear(hidden_states, num_labels)
        
    
    def forward(self, embedding):
        logits = self.classifier(embedding)
        return logits

In [84]:
# create the model
bert_model = VLBertModel(num_labels=4, hidden_states=768)
bert_model = bert_model.to(device)

In [85]:
train_sampler = RandomSampler(train_dataset)        
train_dataloader = DataLoader(dataset=train_dataset,
                    batch_size=32, 
                    sampler=train_sampler)

In [86]:
t_total = len(train_dataloader) * num_train_epochs

optimizer = AdamW(bert_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = get_scheduler(name="cosine", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

criterion = nn.KLDivLoss() #nn.CrossEntropyLoss()

bert_model.train()


start = perf_counter()
for epoch_num in trange(num_train_epochs, desc='Epochs'):
    epoch_total_loss = 0

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc='Batch'):              
        b_labels = batch['summary_embeddings'].to(device)
        b_inputs = batch['labels'].to(device)

        bert_model.zero_grad()  
        b_logits = bert_model(b_inputs)
        m = nn.Sigmoid()
        output = m(input)
        
        loss = criterion(b_logits, b_labels)

        epoch_total_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()


        optimizer.step()
        scheduler.step()
    
    avg_loss = epoch_total_loss/len(train_dataloader)


    print('epoch =', epoch_num)
    print('    epoch_loss =', epoch_total_loss)
    print('    avg_epoch_loss =', avg_loss)
    print('    learning rate =', optimizer.param_groups[0]["lr"])

end = perf_counter()
bert_training_time = end- start
print('Training completed in ', bert_training_time, 'seconds')

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Batch:   0%|          | 0/64042 [00:00<?, ?it/s]

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]], device='cuda:0', dtype=torch.int32)


RuntimeError: mat1 and mat2 must have the same dtype